In [1]:
import pandas as pd
import re
import string
import tqdm
import pickle

In [2]:
texts = pd.read_csv('all.csv', names=['text'])

In [3]:
import unicodedata

In [4]:
def word_rep(word):
    lower = word.lower()
    if lower != word:
        return ('<U>', lower, '</U>')
    return (lower, )
def clean_text(text):
    text = re.sub(r'[ ]*[\n\r]+[ ]*', ' _nl_ ', text)
    text = re.sub(r'[ ]+', '_sp_', text)
    text = re.sub(r'(\W)', r'_sp_\g<1>_sp_', text)
    text = re.sub(r'(_sp_)+', ' ', text)
    text = re.sub(r'\b\d+\b', '<number>', text)
#     text = '<S> ' + text + ' </S>'
    text = text.split() + [None]
    result = []
    count = 1
    for idx, word in enumerate(text[1:], start=1):
        if word == text[idx-1]:
            count += 1
        elif count > 1:
            result.append('<{}>'.format(count))
            result.extend(word_rep(text[idx-1]))
            result.append('</{}>'.format(count))
            count = 1
        else:
            result.extend(word_rep(text[idx-1]))
    return ['<S>'] +  result + ['</S>']

In [7]:
all_text = []
i = 0
for text in tqdm.tqdm_notebook(texts['text']):
    all_text.append(' '.join(clean_text(text)))
#     print(i, end='\r')

In [8]:
import numpy as np

In [16]:
random_text = []
for i in np.random.permutation(np.arange(len(all_text))):
    random_text.append(all_text[i])

In [19]:
len(random_text)

1677662

In [43]:
train = random_text[:-400000]
valid = random_text[-400000:-200000]
test = random_text[-200000:]
print(len(train), len(valid), len(test))

1277662 200000 200000


In [44]:
with open('full/train.pkl', 'wb') as out:
    pickle.dump(train, out)

In [45]:
with open('full/valid.pkl', 'wb') as out:
    pickle.dump(valid, out)
with open('full/test.pkl', 'wb') as out:
    pickle.dump(test, out)

In [26]:
from collections import Counter

In [27]:
counter = Counter()

In [28]:
for line in tqdm.tqdm_notebook(train):
    counter.update(line.split())

In [29]:
idx2word = ['<UNK>'] + [x for x, c in counter.most_common() if c > 10]

In [3]:
import json

In [31]:
word2idx = {}
for idx, w in tqdm.tqdm_notebook(enumerate(idx2word)):
    word2idx[w] = idx + 1

In [32]:
with open('full/word2idx.json', 'w') as out:
    json.dump(word2idx, out)

In [33]:
char_counter = Counter()

In [34]:
for w in tqdm.tqdm_notebook(idx2word):
    char_counter.update(x for x in w)

In [35]:
idx2char = [x for x, c in char_counter.most_common()]
char2idx = {c: i+1 for i, c in enumerate(idx2char)}
with open('full/char2idx.json', 'w') as out:
    json.dump(char2idx, out)

In [37]:
def text2idx(texts):
    unk_idx = word2idx['<UNK>']
    return np.array(
        [word2idx.get(w, unk_idx) for text in tqdm.tqdm_notebook(texts) for w in text.split()]
    )

In [46]:
train_arr = text2idx(train)
test_arr = text2idx(test)
valid_arr = text2idx(valid)

In [47]:
np.save('full/train_word.npy', train_arr)
np.save('full/test_word.npy', test_arr)
np.save('full/valid_word.npy', valid_arr)

In [48]:
print(train_arr.shape)
print(valid_arr.shape)
print(test_arr.shape)

(391717938,)
(61479417,)
(61404695,)


In [4]:
with open('full/char2idx.json', 'r') as out:
    char2idx = json.load(out)
with open('full/word2idx.json', 'r') as out:
    word2idx = json.load(out)

In [5]:
with open('full/train.pkl', 'rb') as inp:
    train = pickle.load(inp)
with open('full/valid.pkl', 'rb') as inp:
    valid = pickle.load(inp)
with open('full/test.pkl', 'rb') as inp:
    test = pickle.load(inp)

In [6]:
def text2charidx(texts):
    word2char = {w: [char2idx[c] for c in w] for w in word2idx}
    unk_list = word2char['<UNK>']
    return np.array(
        [word2char[w] if w in word2idx else unk_list for text in tqdm.tqdm_notebook(texts) for w in text.split()]
    )

In [ ]:
train_char = text2charidx(train)
test_char = text2charidx(test)
valid_char = text2charidx(valid)

In [ ]:
np.save('full/train_char.npy', train_char)
np.save('full/test_char.npy', test_char)
np.save('full/valid_char.npy', valid_char)

In [ ]:
a = ' '.join(idx2word[i-1] for i in test_arr[:10000])
b = ' '.join(''.join(idx2char[i-1] for i in w) for w in test_char[:10000])
a == b